In [1]:
import modin.pandas as pd
from modin.config import NPartitions

print("Modin partitions:", NPartitions.get())

from modin.config import ProgressBar
from tqdm import tqdm

ProgressBar.enable()

from shapely.geometry import Point

Modin partitions: 28


In [2]:
# Load dataset.
tweets = pd.read_json("tweets.jsonl", lines=True)

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2023-07-19 14:10:54,431	INFO worker.py:1636 -- Started a local Ray instance.


In [3]:
# Set main index.
tweets.set_index("id", inplace=True)

In [4]:
# Check assumptions on geographical data.
assert (tweets["geo"].isna() == tweets["coordinates"].isna()).all()
assert tweets["geo"].map(lambda x: x["type"] == "Point", "ignore").all()
assert tweets["coordinates"].map(lambda x: x["type"] == "Point", "ignore").all()
assert tweets["geo"].map(lambda x: len(x["coordinates"]) == 2, "ignore").all()
assert tweets["coordinates"].map(lambda x: len(x["coordinates"]) == 2, "ignore").all()
assert (
    tweets["geo"].dropna().map(lambda x: x["coordinates"]).map(reversed).map(list)
    == tweets["coordinates"].dropna().map(lambda x: x["coordinates"])
).all()

# Normalize geographical data.

tweets["point"] = tweets["coordinates"].map(
    lambda x: Point(x["coordinates"]).wkt, "ignore"
)
tweets.drop(columns=["geo", "coordinates"], inplace=True)
tweets["place_id"] = tweets["place_id"].fillna("0").apply(int, base=16).astype("uint64")

Estimated completion of line 2:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 3:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 4:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 5:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 6:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 8:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 9:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 10:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 14:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 18:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [5]:
from lib import entities

# Check assumptions on textual data.
assert (
    tweets["entities"]
    .map(lambda x: all(k in entities.ENTITY_DELIMITERS_BY_KEY.keys() for k in x.keys()))
    .all()
)

# Normalize textual data.
tweets["full_text_with_entities"] = tweets[["full_text", "entities"]].apply(
    entities.embed_entity_annotations, axis=1
)
tweets.drop(columns=["full_text", "entities"], inplace=True)

Estimated completion of line 6:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 7:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [8]:
# Dump to parquet. The dataset ain't that big, so we force writing a single file.
tweets._to_pandas().to_parquet("tweets.parquet")